<a href="https://colab.research.google.com/github/theMADAIguy/STOCK-PREDICTION-USING-SVM-REGRESSION/blob/master/text_fastText_pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchtext
from torchtext import data, datasets
import random

SEED = 1234
BATCH_SIZE=64
EPOCHS=10

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/
PATH = './text_fast_test_model.pt'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [0]:
def generate_ngram(x):
  n_grams=set(zip(*[x[i:] for i in range(2)]))
  for gram in n_grams:
    x.append(' '.join(gram))
  return x


In [4]:
TEXT=data.Field(tokenize='spacy', preprocessing=generate_ngram)
LABEL=data.LabelField(dtype=torch.float)

trainSet, testSet=datasets.IMDB.splits(TEXT, LABEL)
trainSet, validSet=trainSet.split(random_state=random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.8MB/s]


In [5]:
TEXT.build_vocab(trainSet, max_size=MAX_SIZE_VOCAB, vectors='glove.6B.100d', unk_init=torch.Tensor.normal_)
LABEL.build_vocab(trainSet)

trainIterator, validIterator, testIterator=data.BucketIterator.splits((trainSet, validSet, testSet), batch_size=BATCH_SIZE, device=device)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
100%|█████████▉| 399728/400000 [00:19<00:00, 20807.95it/s]

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class fastText(nn.Module):
  def __init__(self, embedding_dim, output_dim, vocab_size, pad_idx):
    super().__init__()

    self.embedding=nn.Embedding(vocab_size, embedding_dim, pad_idx)
    self.fc=nn.Linear(embedding_dim, output_dim)
    
  def forward(self, text):
    embedded=self.embedding(text)
    embedded=embedded.permute(1,0,2)
    pooled=F.avg_pool2d(embedded, (embedded.shape[1],1)).squeeze(1)
    return self.fc(pooled)


In [24]:
INPUT_DIM=len(TEXT.vocab)
EMBEDDING_DIM=100
OUTPUT_DIM=1
MAX_SIZE_VOCAB=25004
PAD_IDX=TEXT.vocab.stoi[TEXT.pad_token]

model=fastText(EMBEDDING_DIM, OUTPUT_DIM, MAX_SIZE_VOCAB, PAD_IDX)
print(model)

fastText(
  (embedding): Embedding(25004, 100, padding_idx=1)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)


In [25]:
pre_trained_embedding=TEXT.vocab.vectors
print(pre_trained_embedding.shape)
model.embedding.weight.data.copy_(pre_trained_embedding)
UNK_IDX=TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX]=torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX]=torch.zeros(EMBEDDING_DIM)

torch.Size([25004, 100])


In [26]:
import torch.optim as optim
optimizer=optim.Adam(model.parameters())
criterion=nn.BCEWithLogitsLoss()

model.to(device)
criterion.to(device)


BCEWithLogitsLoss()

In [0]:
def accuracy(prediction, y):

  rounded_pred=torch.round(torch.sigmoid(prediction))
  correct=(rounded_pred==y).float()
  acc=correct.sum()/len(correct)
  return acc

In [0]:
def train(model, iterator, optimizer, criterion):

  epoch_loss=0.0
  epoch_acc=0.0

  model.train()

  for batch in iterator:

    optimizer.zero_grad()
    output=model(batch.text).squeeze(1)
    
    acc=accuracy(output, batch.label)
    loss=criterion(output, batch.label)

    loss.backward()
    optimizer.step()

    epoch_acc+=acc.item()
    epoch_loss+=loss.item()
  return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [0]:
def evaluate(model, iterator, criterion):

  epoch_loss=0.0
  epoch_acc=0.0

  model.eval()

  with torch.no_grad():
      
    for batch in iterator:

      output=model(batch.text).squeeze(1)
      
      acc=accuracy(output, batch.label)
      loss=criterion(output, batch.label)

      epoch_acc+=acc.item()
      epoch_loss+=loss.item()
  return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [38]:
best_valid_loss=float('inf')

for epoch in range(EPOCHS):

  train_loss, train_acc=train(model, trainIterator, optimizer, criterion)
  valid_loss, valid_acc=evaluate(model, validIterator, criterion)

  if best_valid_loss > valid_loss:
    best_valid_loss=valid_loss
    torch.save(model.state_dict(), PATH)
  
  print("Epoch: ", epoch, "Train Loss: ", train_loss, "Valid Loss: ", valid_loss, "Train Acc: ", train_acc, "Valid Acc: ", valid_acc)



Epoch:  0 Train Loss:  0.49692417583326354 Valid Loss:  0.3916273618401107 Train Acc:  0.8413875131276403 Valid Acc:  0.8352754237288136
Epoch:  1 Train Loss:  0.430133659202252 Valid Loss:  0.37490928728701706 Train Acc:  0.8701039493954095 Valid Acc:  0.8542549433344502
Epoch:  2 Train Loss:  0.38028284711559324 Valid Loss:  0.37824867627883363 Train Acc:  0.8852238659441036 Valid Acc:  0.8677612992666536
Epoch:  3 Train Loss:  0.34082594834757546 Valid Loss:  0.38896420774823526 Train Acc:  0.8975821167883211 Valid Acc:  0.8747793077412298
Epoch:  4 Train Loss:  0.31016405171503986 Valid Loss:  0.40462483528812054 Train Acc:  0.9060870698333656 Valid Acc:  0.8796786721480094
Epoch:  5 Train Loss:  0.28328361867988194 Valid Loss:  0.41760733477392437 Train Acc:  0.9132641423357665 Valid Acc:  0.8847104518090264
Epoch:  6 Train Loss:  0.2636210916795 Valid Loss:  0.43142836730358963 Train Acc:  0.9187223019391081 Valid Acc:  0.8860346043513994
Epoch:  7 Train Loss:  0.2420231134623941

In [40]:
test_loss, test_acc = evaluate(model, testIterator, criterion)
print("Test Loss: ", test_loss, "Test Acc", test_acc)

Test Loss:  0.5009233776451377 Test Acc 0.8902093989160055
